In [19]:
import pandas as pd

# 따로 깔지 않아도 됌
import os
import sys
import glob
import geopandas as gpd
import fiona
import shapely 
import shapefile

pd.set_option('display.max_columns',50)
pd.set_option('display.max_colwidth',30)

In [21]:
"""
픽스할 공간은 -> 승강장
승강장은 논문에서 봤을 때 스크린 도어 앞, 병목현상
불가용공간, 교차흐름이 발생하는 공간 -> 비율이나, 추가 논리 설정
우리 있는 정보

1단계) 서론
-> 섬식 승강장, 무슨 승강장인지 -> 호선 별 승강장 타입 매칭
서울교통공사 지하철 역사건축정보 -> 섬식승강장,
지하철 주요설계기준(건축분야)이용해서 가용승강장면적 (가설)

(대합실 배제)
2단계) 본론
-> 군중압력, 밀집도(혼잡도),하차인원 고려한 산식 설정
산식 기반, 위험도를 임의로 생성 
-> 뭐가 다른데랑 다르길래?
# 교통안전시설물, 스크린도어 psd 

3단계) 결론
어떻게 개선을 할거냐?

본론에 추가를 하면 좋을 것
같은 수준의 혼잡도를  가지고 있어도
편의시설이 부족한 곳의 교통약자 기준으로는 체감위험도가  달라지니깐

교통약자에 대한 추가 체감위험도에 대한 변화율
수식 변수 * 변수 + 상수(안전시설물) -> 표준조례, 기준


혼잡도 2개년형, 승하차인원 5개년형 -> 경향성을 예상해서, 미리 선제적으로 해당 역에 전광판에 띄우는 방식으로 위험도를 설명

출입구에 승강장 내부 혼잡도 미리 보여주는거


프로토타입도 api 콜 받아서 만들어보기
"""

'\n픽스할 공간은 -> 승강장\n승강장은 논문에서 봤을 때 스크린 도어 앞, 병목현상\n불가용공간, 교차흐름이 발생하는 공간 -> 비율이나, 추가 논리 설정\n우리 있는 정보\n\n1단계) 서론\n-> 섬식 승강장, 무슨 승강장인지 -> 호선 별 승강장 타입 매칭\n서울교통공사 지하철 역사건축정보 -> 섬식승강장,\n지하철 주요설계기준(건축분야)이용해서 가용승강장면적 (가설)\n\n(대합실 배제)\n2단계) 본론\n-> 군중압력, 밀집도(혼잡도),하차인원 고려한 산식 설정\n산식 기반, 위험도를 임의로 생성 \n-> 뭐가 다른데랑 다르길래?\n# 교통안전시설물, 스크린도어 psd \n\n3단계) 결론\n어떻게 개선을 할거냐?\n\n같은 수준의 혼잡도를  가지고 있어도\n편의시설이 부족한 곳의 교통약자 기준으로는 체감위험도가  달라지니깐\n\n교통약자에 대한 추가 체감위험도에 대한 변화율\n수식 변수 * 변수 + 상수(안전시설물) -> 표준조례, 기준\n\n\n\n출입구에 승강장 내부 혼잡도 미리 보여주는거\n\n\n프로토타입도 api 콜 받아서 만들어보기\n'

In [ ]:
### 컴공 가도,, 잘해야되는게 꼭 있는데
### 그게 이제, 파일 입출력 관리, -> os ,sys , glob shutil
### 그거랑 네트워크 , 성능문제 -> requests bsp4 selenium
### 백단 -> spring, redis 
### 네트워크 -> 보안, github 팀단위 작업
### 예를 들어서 성윤이가 너랑 같이 작업한 거에서 새로운 파일을 만듦
### 너도 만들었어
### 공유된 곳에서 서로 다른 로컬 pc에서 각기 다른 파일이 생긴거잠ㄴㅎ아?
### 이걸 묶어줘야하니까 branch 에서 merge도 해야하고 상대파일도 받아와야함 -> pull

### 컴공 4대 cs -> 네트워크 ,알고리즘, 데이터베이스, 운영체제
### 네트워크,알고리즘 > 데베 > 운영체제

In [ ]:
'''
혼잡도를 해석하는 방식
-> 혼잡도 csv 해석이 되는거지
문제가 생길 수 있는 시간 -> 같은 시간에 (현재역 혼잡도 - 전역 혼잡도) * 160 (인원수) > 군중밀집계수 * 스크린 도어 앞 승강장 가용면적
이게 자주 발생하는 시간의 해당 역에 전광판? 

-> 어떻게 시간 당 승하차인원을 끼얹나 (1시단 단위)
30분단위잖아
보정치로 해당 시간 내의 혼잡도가 상수로 들어가야함
5:30 의 현재역의 혼잡도가 18 6:00 의 혼잡도가 12였다면
300명이면 , 180명 120명 탓겠지
같은 역 5시~6시 300명 타면 , 해당 시간 승차인원 * (해당시간 승차시의 혼잡도(A or B)/전체(A+B)혼잡도 로 승차인원을 한번더 쪼갠 테이블로 승하차 인원 변환  
'''

In [ ]:
"""
해당 역이 환승역일 때 성립되는 모델

상봉(7호선,경춘선)

개찰구에 승차를 1000명이 찍었다

1시간 동안 열차가 10번 왔고, 30명씩 탔어

현재 개찰구 이후 공간에서 700명이 잔류

하차 태그를 찍고 개찰구를 벗어난 사람이 200명

500명이 잔류 -> 환승인원

환승역의 다음 역의 혼잡도가 올라간다.

"""

In [82]:
# csv 묶어놓기
# glob.glob 표현식에 맞는 파일들만 concat
# * -> wild card -> 원카드로 치면 조커 -> 뭐든지 되는거
file_names = glob.glob("./data_origin/CARD_*.csv")


# 빈 데이터 프레임
res = pd.DataFrame() # result (줄인거)

for file_name in file_names:
    # 매번  월 별 데이터를 가져오는 거지
    tmp = pd.read_csv(file_name, sep = ',',encoding= 'cp949')
    # 최종적으로 붙일거에  자꾸 concat
    res = pd.concat([res,tmp])
res

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20170501,4호선,수유(강북구청),43727.0,42394.0,20170504
1,20170501,4호선,미아(서울사이버대학),18918.0,17349.0,20170504
2,20170501,4호선,미아사거리,30669.0,29788.0,20170504
3,20170501,4호선,길음,25919.0,25997.0,20170504
4,20170501,4호선,성신여대입구(돈암),27949.0,25741.0,20170504
...,...,...,...,...,...,...
18300,20190731,우이신설선,북한산보국문,5662.0,5145.0,20190803
18301,20190731,우이신설선,정릉,4282.0,3768.0,20190803
18302,20190731,우이신설선,성신여대입구(돈암),3465.0,3854.0,20190803
18303,20190731,우이신설선,보문,1500.0,1599.0,20190803


In [ ]:
file_names = glob.glob('./')

In [9]:
sample = gpd.read_file('data_origin/AL_11140_D198_20220930/AL_11140_D198_20220930.shp',encoding='cp949')

In [15]:
sample.columns

Index(['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20',
       'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30',
       'A31', 'A32', 'A33', 'A34', 'A35', 'geometry'],
      dtype='object')

In [20]:
sample[sample.A25 == '운수시설']

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32,A33,A34,A35,geometry
835,61152803,1968197313634507461500000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3708,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,72148.00,342.20,28190.54,39.0,NaN,51,일반목구조,08000,운수시설,08999,기타운수시설,2,상업용,6.2,2.0,0.0,None,1968-09-14,2022-09-29,POLYGON ((197318.876 45073...
837,61140026,1979197238234502706200000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3705,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,23272.28,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08007,철도시설,2,상업용,0.0,8.0,2.0,None,1979-08-08,2022-09-29,POLYGON ((197225.976 45022...
842,61141379,1984197329284505864700000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3716,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,605.45,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08007,철도시설,2,상업용,0.0,2.0,0.0,None,1984-02-23,2022-09-29,POLYGON ((197322.655 45060...
843,61141842,1967197300364507625700000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3711,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,72148.00,960.00,28190.54,39.0,1.0,21,철근콘크리트구조,08000,운수시설,08999,기타운수시설,2,상업용,3.6,2.0,0.0,None,1967-07-23,2022-09-29,POLYGON ((197294.579 45074...
844,61145484,0000197300774506405700000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3703,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,742.26,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08999,기타운수시설,2,상업용,0.0,2.0,0.0,None,None,2022-09-29,POLYGON ((197295.514 45065...
845,61144303,1937197238444505990300000000,1114012000101220000,1114012000,서울특별시 중구 봉래동2가,1,일반,122,3704,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,33.06,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08999,기타운수시설,2,상업용,0.0,1.0,0.0,None,1937-10-18,2022-09-29,POLYGON ((197237.216 45059...
4985,61141688,1993199833264515197000000000,1114015000101780001,1114015000,서울특별시 중구 을지로4가,1,일반,178-1,11554,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,104.92,56.23,204.65,160.0,53.0,21,철근콘크리트구조,08000,운수시설,08000,운수시설,6,문교사회용,9.7,3.0,1.0,None,1993-04-12,2022-09-29,POLYGON ((199836.128 45151...
4991,61140764,1966199814784515276800000000,1114015000101830003,1114015000,서울특별시 중구 을지로4가,1,일반,183-3,11563,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,586.35,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08000,운수시설,6,문교사회용,0.0,4.0,0.0,None,1966-11-09,2022-09-29,POLYGON ((199810.831 45152...
5745,61147371,1969199383164515414500000000,1114015500102780000,1114015500,서울특별시 중구 을지로3가,1,일반,278,13189,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,733.24,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08000,운수시설,6,문교사회용,0.0,4.0,1.0,None,1969-10-30,2022-09-29,POLYGON ((199382.530 45152...
5814,61154931,1956199185254515366300000000,1114015500103150001,1114015500,서울특별시 중구 을지로3가,1,일반,315-1,13304,1,일반건축물,2,일반건축물대방,None,None,0,주건축물,0.00,0.00,579.73,0.0,0.0,21,철근콘크리트구조,08000,운수시설,08000,운수시설,6,문교사회용,0.0,5.0,1.0,None,1956-09-15,2022-09-29,POLYGON ((199191.044 45153...


In [83]:
res.to_csv('./data_processed/passenger_per_hour.csv',encoding='utf-8')

In [84]:
res = pd.DataFrame()
for i in glob.glob("./data_origin/서울교통공사_엘리베이터*.csv"):
    tmp = pd.read_csv(i, sep = ',',encoding= 'cp949')
    # 최종적으로 붙일거에  자꾸 concat
    res = pd.concat([res,tmp])

res


,연번,호선,역명,장비,호기,승강기번호,운행구간,설치위치,비고
0,1,1,서울(1),E/L,내부#1,0074-205,B2(승)~B1(대),섬식(상) 5-1,NaN
1,2,1,서울(1),E/L,외부#1,0104-656,B1(대)~지상,2번 출구측,NaN
2,3,1,서울(1),E/L,외부#2,0105-867,B1(대)~지상,4번 출구측,NaN
3,4,1,서울(1),E/L,외부#3,0073-979,B1(대)~지상,3번 출구측,NaN
4,5,1,시청(1),E/L,내부#1,0002-888,B2(승)~B1(대),상행 3-2,NaN
...,...,...,...,...,...,...,...,...,...
877,878,8,수진,E/L,2,2116-590,B1-B2,내부,NaN
878,879,8,수진,E/L,3,2212-625,B1-F1,외부,NaN
879,880,8,모란,E/L,1,2116-583,B1-B3,내부,NaN
880,881,8,모란,E/L,2,2116-584,B2-B3,내부,NaN


In [85]:
res.to_csv('./data_processed/station_elevator_current.csv',encoding='utf-8')
# 컴퓨터 연산에는 3가지가 있는데 read -> 파일 불러오는 거
# concat processing
# write -> 저장 

In [72]:
# 중복체크가 이런 문법으로 잘 안되는 데이터셋? -> 
res[res.duplicated(['승강기번호']) == True]

# 0074-205 -> 74 205 

res['새컬럼'] = ''

In [75]:
res['새컬럼'] = [i*3 for i in range(1731)]

In [80]:
res['승강기번호'].iloc[0][:4]  # 1번 컬럼
res['승강기번호'].iloc[0][5:]  # 2번 컬럼

'205'

In [76]:
res

,연번,호선,역명,장비,호기,승강기번호,운행구간,설치위치,비고,새컬럼
0,1,1,서울(1),E/L,내부#1,0074-205,B2(승)~B1(대),섬식(상) 5-1,NaN,0
1,2,1,서울(1),E/L,외부#1,0104-656,B1(대)~지상,2번 출구측,NaN,3
2,3,1,서울(1),E/L,외부#2,0105-867,B1(대)~지상,4번 출구측,NaN,6
3,4,1,서울(1),E/L,외부#3,0073-979,B1(대)~지상,3번 출구측,NaN,9
4,5,1,시청(1),E/L,내부#1,0002-888,B2(승)~B1(대),상행 3-2,NaN,12
...,...,...,...,...,...,...,...,...,...,...
877,878,8,수진,E/L,2,2116-590,B1-B2,내부,NaN,5178
878,879,8,수진,E/L,3,2212-625,B1-F1,외부,NaN,5181
879,880,8,모란,E/L,1,2116-583,B1-B3,내부,NaN,5184
880,881,8,모란,E/L,2,2116-584,B2-B3,내부,NaN,5187


In [65]:
res.sample(5) 
# PK -> 무결성,비중복성 unique key -> 중복을 허용하지 않는다 뜻

,연번,호선,역명,장비,호기,승강기번호,운행구간,설치위치,비고
827,828,8,암사,E/L,2,0066-067,B1-B2,내부,NaN
826,827,8,암사,E/L,1,0066-066,B1-B2,내부,NaN
323,324,4,성신여대입구,E/L,외부#2,0028-003,B1(대)~지상,3번 출구측,NaN
168,169,2(기타),신정차량기지,E/L,2,0029-269,F1~5,관리동,NaN
442,443,5,충정로,E/L,2,0102-186,F1-B1,8번출구,NaN


In [ ]:
res.drop_duplicates(['컬럼1','컬럼2'])

In [81]:
res.to_csv('./data_processed/test.csv',encoding='utf-8')

In [61]:
# DataFrame 여러가지 메소드

res[res.duplicated() == True]
# 2개년 형 엘리베이터 를 concat했는데, 198개의 중복이 나왔다 -> 이런 정보는 걷어내야함

res['역명'].unique()

# 너가 해결해야할 이슈는 -> 나처럼 glob.glob 써서 원하는파일을 불러오기
# 이게 안되면 os module 에도 있음 -> 이거를 공부해
# 요거 하다가 정 안되면,, 머리에 남겨두고, 데이터 셋 더 찾아보고 나한테 질문 
# 요런 데이터도 쓸 수 있을까? 원본 주고 

# 만약에 이게 된다.. 그러면 그 다음으로 넘어가는거지 dataframe 도 만져보고 전처리
# 짬이 나면, 데이터셋 더 찾고?

array(['서울(1)', '시청(1)', '종각', '종로3가(1)', '종로5가', '동대문(1)', '동묘앞',
       '신설동(1)', '제기동', '청량리(1)', '시청(2)', '을지로입구', '을지로3가(2)', '을지로4가',
       '동대문역사문화공원(2)', '신당', '상왕십리', '왕십리', '한양대', '뚝섬', '성수', '신답',
       '신답별관', '용두', '인재개발원', '신설동(2)', '건대입구', '구의', '강변', '잠실나루', '잠실',
       '잠실새내', '종합운동장', '삼성', '선릉', '역삼', '강남', '교대(2)', '서초', '방배',
       '사당(2)', '낙성대', '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림',
       '대림별관', '신도림', '도림천', '양천구청', '신정차량기지', '신정네거리', '문래', '영등포구청',
       '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '지축', '구파발', '연신내',
       '불광', '녹번', '홍제', '무악재', '독립문', '경복궁', '안국', '종로3가(3)', '을지로3가(3)',
       '동대입구', '약수', '금호', '옥수', '압구정', '신사', '잠원', '고속터미널', '교대(3)',
       '남부터미널', '양재', '매봉', '대치', '학여울', '대청', '일원', '수서', '가락시장', '경찰병원',
       '오금', '당고개', '상계', '노원', '창동', '쌍문', '수유', '미아', '미아사거리', '길음',
       '성신여대입구', '한성대입구', '혜화', '동대문(4)', '동대문역사문화공원(4)', '동대문별관',
       '충무로(4)', '명동', '회현', '서울(4)', '숙대입구', '삼각지', '신용산', '이촌', '동작',
       

In [ ]:
# 지금 우리가 가지고 있는 정보 concat 

In [16]:
type(file_names) 
# -> iterable
file_names[:3]

['./data_origin/CARD_SUBWAY_MONTH_201705.csv',
 './data_origin/CARD_SUBWAY_MONTH_201711.csv',
 './data_origin/CARD_SUBWAY_MONTH_201710.csv']

In [17]:
res.to_csv('./data_processed/passenger_per_hour.csv',encoding='cp949')

In [20]:
pd.read_csv('./data_processed/passenger_per_hour.csv',encoding='cp949')

,Unnamed: 0,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,0,20170501,4호선,수유(강북구청),43727.0,42394.0,20170504
1,1,20170501,4호선,미아(서울사이버대학),18918.0,17349.0,20170504
2,2,20170501,4호선,미아사거리,30669.0,29788.0,20170504
3,3,20170501,4호선,길음,25919.0,25997.0,20170504
4,4,20170501,4호선,성신여대입구(돈암),27949.0,25741.0,20170504
...,...,...,...,...,...,...,...
424021,18300,20190731,우이신설선,북한산보국문,5662.0,5145.0,20190803
424022,18301,20190731,우이신설선,정릉,4282.0,3768.0,20190803
424023,18302,20190731,우이신설선,성신여대입구(돈암),3465.0,3854.0,20190803
424024,18303,20190731,우이신설선,보문,1500.0,1599.0,20190803


In [6]:
cp17 = pd.read_csv('./서울교통공사_혼잡도_20171231.csv',encoding = 'cp949') # utf-8
cp19 = pd.read_csv('./서울교통공사_혼잡도_20191231.csv',encoding = 'cp949')

In [14]:
p = pd.read_csv('./CARD_SUBWAY_MONTH_201701.csv',encoding='cp949')
len(p['사용일자'].unique()) # -> 1달 31일 준다는 소리

31

In [ ]:
# 찾은 데이터셋이 3 종류 -> 혼잡도(4개) 월별 승차인원(2개년 -> 시점을 일치시켜야하니까)

# 혼잡도 데이터를 17년도,19년도,13년도,11년도가 있으니까 예상을 해볼까
'''
데이터가 의미가 있으려면, 기본적으로 시점이 일치해야되는데, 지금 승하차인원은은 17,19밖에 없음 혼잡도가 승하차랑 연계될려면
혼잡도가 데이터가 반토막나니까,, 그게 좀 아쉬워서 문득 든 생각이고

혼잡도라는게 매 년마다 상이한 차이가 발생하진 않잖아?
그럼 11년,13년에 발생한 혼잡도를 기반으로 학습을 시키면, (머신러닝) -> 17년 19년도에 혼잡한 곳을 예측할 수 있지않을까..
아니면은 22년에 ,,, 4개년의 혼잡도랑 승객수랑 이런걸 학습시켜서, 일정 인원 이상이 승차하는 순간부터 혼잡도가 올라가는 구간이 기하급수? 처럼 될거고
그럼 군중압력 많이 생길거고,, 이것도 좀 낑겨서 어떻게 넣으면 좋을 것 같은데... 

지하철 외부 출입구 계단 너비,폭 내부 계단 너비,폭 시간대 별 승하차 인원정보, 시간대별 열차 내 혼잡도, 장애인 외출목적통계, 논문? 실태조사(우리 방향성 잡아주는데 도움이 될 거고), 군중 압력이론 관련 내용, 지하철 역사 내 수용 가능인구 혹은 수용 가능 면적
첨부 파일 형식: acrobat
5.723-4-.pdf
645.50 KB
장애인들이 주로 사용하는 장비인 휠체어가 일반 성인보다 차지하는 면적도 크니까, 모델 별 평균을 내도 되고 성인이 차지하는 면적에 보정치를 넣어도 될 것 같음
'''

'''
https://data.kric.go.kr/rips/
https://www.data.go.kr/data/15044440/fileData.do

'''

In [ ]:
'''
찾은 데이터 -> 시간대별 열차 내 혼잡도, 시간대 별 승하차 인원정보

찾아야하는 데이터 -> 
'''

In [10]:
cp17.columns # 30분 단위 컬럼, 12시30분 이후 안뜸
cp17['호선'].unique() # 9호선이 없음
cp19['호선'].unique() # 9호선이 없음

array(['1호선', '2호선', '성수지선', '신정지선', '3호선', '4호선', '5호선', '6호선', '7호선',
       '8호선'], dtype=object)